In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e5/sample_submission.csv
/kaggle/input/playground-series-s3e5/train.csv
/kaggle/input/playground-series-s3e5/test.csv


In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 19.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=c2ade60ec7db13a10d89e2f5280e3ef0da36f67ed5e6a626a56f2843cc137fab
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Note: you may need to restart the kernel to use updated packages.


**IMPORTING ALL THE NECESSARY FILES**

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

from pyspark.ml import Pipeline
#from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [4]:
from pyspark import sql
from pyspark.sql import SparkSession

**CREATING SPARK SESSION**

In [5]:
spark = SparkSession.builder.appName("wine_prediction").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/15 11:15:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
spark

#![](http://)

![WINE](https://images.unsplash.com/photo-1569153482031-a3cebdedf294?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxzZWFyY2h8Mnx8d2luZSUyMGdsYXNzfGVufDB8fDB8fA)

**READING THE DATA FILES**

In [7]:
df=spark.read.csv('/kaggle/input/playground-series-s3e5/train.csv',inferSchema=True, header=True)

In [8]:
test_main=spark.read.csv('/kaggle/input/playground-series-s3e5/test.csv',inferSchema=True, header=True)

In [9]:
sub=spark.read.csv('/kaggle/input/playground-series-s3e5/sample_submission.csv',inferSchema=True, header=True)

In [10]:
df.show()

+---+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
| Id|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+---+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|  0|          8.0|             0.5|       0.39|           2.2|    0.073|               30.0|                39.0|0.99572|3.33|     0.77|   12.1|      6|
|  1|          9.3|             0.3|       0.73|           2.3|    0.092|               30.0|                67.0|0.99854|3.32|     0.67|   12.8|      6|
|  2|          7.1|            0.51|       0.03|           2.1|    0.059|                3.0|                12.0| 0.9966|3.52|     0.73|   11.3|      7|
|  3|          8.1|            0.87|       0.22|           2.6|    0.084|   

**showing the data in pandas format**

In [11]:
df.toPandas()

,Id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0,8.0,0.50,0.39,2.20,0.073,30.0,39.0,0.99572,3.33,0.77,12.1,6
1,1,9.3,0.30,0.73,2.30,0.092,30.0,67.0,0.99854,3.32,0.67,12.8,6
2,2,7.1,0.51,0.03,2.10,0.059,3.0,12.0,0.99660,3.52,0.73,11.3,7
3,3,8.1,0.87,0.22,2.60,0.084,11.0,65.0,0.99730,3.20,0.53,9.8,5
4,4,8.5,0.36,0.30,2.30,0.079,10.0,45.0,0.99444,3.20,1.36,9.5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051,2051,6.6,0.31,0.13,2.00,0.056,29.0,42.0,0.99388,3.52,0.87,12.0,7
2052,2052,9.7,0.59,0.21,1.80,0.079,27.0,65.0,0.99745,3.14,0.58,9.4,5
2053,2053,7.7,0.43,0.42,1.70,0.071,19.0,37.0,0.99258,3.32,0.77,12.5,8
2054,2054,9.1,0.50,0.00,1.75,0.058,5.0,13.0,0.99670,3.22,0.42,9.5,5


**CHECKING THE NULL VALUES**

In [12]:
from pyspark.sql.functions import col,isnan, when, count

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+---+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
| Id|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density| pH|sulphates|alcohol|quality|
+---+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+
|  0|            0|               0|          0|             0|        0|                  0|                   0|      0|  0|        0|      0|      0|
+---+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+---+---------+-------+-------+



In [13]:
df=df.drop('Id')

In [14]:
data=df.select('fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol','quality')
data.show(10)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          8.0|             0.5|       0.39|           2.2|    0.073|               30.0|                39.0|0.99572|3.33|     0.77|   12.1|      6|
|          9.3|             0.3|       0.73|           2.3|    0.092|               30.0|                67.0|0.99854|3.32|     0.67|   12.8|      6|
|          7.1|            0.51|       0.03|           2.1|    0.059|                3.0|                12.0| 0.9966|3.52|     0.73|   11.3|      7|
|          8.1|            0.87|       0.22|           2.6|    0.084|               11.0|           

**CHCEKING DATA TYPES**

In [15]:
df.dtypes

[('fixed acidity', 'double'),
 ('volatile acidity', 'double'),
 ('citric acid', 'double'),
 ('residual sugar', 'double'),
 ('chlorides', 'double'),
 ('free sulfur dioxide', 'double'),
 ('total sulfur dioxide', 'double'),
 ('density', 'double'),
 ('pH', 'double'),
 ('sulphates', 'double'),
 ('alcohol', 'double'),
 ('quality', 'int')]

**CREATING VECTORS**

In [16]:
featVect = VectorAssembler(inputCols=['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol'], outputCol="features")
data=featVect.transform(df)

**Split the Data**

I will use 70% of the data for training, and reserve 30% for testing. In the testing data, the label column is renamed to trueLabel so I can use it later to compare predicted labels with known actual values.

In [17]:

splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 1428  Testing Rows: 628


**Define the Pipeline**


A pipeline consists of a series of transformer and estimator stages that typically prepare a DataFrame for modeling and then train a predictive model. In this case, you will create a pipeline with seven stages:


A StringIndexer estimator that converts string values to indexes for categorical features
A VectorAssembler that combines categorical features into a single vector
A VectorIndexer that creates indexes for a vector of categorical features
A VectorAssembler that creates a vector of continuous numeric features
A MinMaxScaler that normalizes continuous numeric features
A VectorAssembler that creates a vector of categorical and continuous features
A DecisionTreeClassifier that trains a classification model.

In [18]:
#strIdx = StringIndexer(inputCol = "Carrier", outputCol = "CarrierIdx")
#catVect = VectorAssembler(inputCols = ["CarrierIdx", "DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID"], outputCol="catFeatures")
#catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures")
#numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
#minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")
#featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")
lr = LogisticRegression(labelCol="quality",featuresCol="features",maxIter=10,regParam=0.3)
#dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
#pipeline = Pipeline(stages=[strIdx, catVect, catIdx, numVect, minMax, featVect, lr])


In [19]:
train.show()

+-------------+----------------+-----------+--------------+------------------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|         chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|            features|
+-------------+----------------+-----------+--------------+------------------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|          5.0|            0.43|       0.24|           2.5|0.0579999999999999|               11.0|                35.0|0.99258|3.22|     0.55|   12.2|      8|[5.0,0.43,0.24,2....|
|          5.0|             0.5|        0.0|           1.7|             0.066|               19.0|                48.0|0.99294|3.78|     0.66|   12.1|      6|[5.0,0.5,0.0,1.7,...|
|          5.0|             0.5|        0.0|           1.8|             0.048|               15.0|  

In [20]:
test.show()

+-------------+----------------+-----------+--------------+------------------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|         chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|            features|
+-------------+----------------+-----------+--------------+------------------+-------------------+--------------------+-------+----+---------+-------+-------+--------------------+
|          5.0|            0.32|        0.0|           1.9|             0.044|                8.0|                18.0|0.99235|3.78|     0.73|   12.4|      6|[5.0,0.32,0.0,1.9...|
|          5.0|             0.4|       0.01|           2.1|             0.118|               19.0|                58.0| 0.9976|3.54|     0.88|   12.4|      6|[5.0,0.4,0.01,2.1...|
|          5.0|            0.46|       0.11|           1.6|0.0559999999999999|               12.0|  

**RANDOM FOREST MODEL**

In [21]:
from pyspark.ml.classification import RandomForestClassifier
rf=RandomForestClassifier(labelCol='quality',featuresCol='features',maxDepth=5)

In [22]:
model1=lr.fit(train)

In [23]:
predictions1=model1.transform(test)

In [24]:
evaluator1=MulticlassClassificationEvaluator(labelCol='quality',predictionCol='prediction',metricName='accuracy')

NameError: name 'MulticlassClassificationEvaluator' is not defined

In [ ]:
accuracy=evaluator.evaluate(predictions)
print(accuracy)

In [ ]:
model=rf.fit(train)

In [ ]:
predictions=model.transform(test)

**EVALUATION METRICS**

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator=MulticlassClassificationEvaluator(labelCol='quality',predictionCol='prediction',metricName='accuracy')

**CHECKING THE ACCURACY**

In [ ]:
accuracy=evaluator.evaluate(predictions)

In [ ]:
print(accuracy)

In [ ]:
test_main=test_main.drop("id")

In [ ]:
test_main.show()

In [ ]:
featVect1 = VectorAssembler(inputCols=['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol'], outputCol="features")
data1=featVect1.transform(test_main)

In [ ]:
data1.show()

**PREDICTIONS**

In [ ]:
pred=model.transform(data1)

In [ ]:
pred.show()

In [ ]:
pred.count()

In [ ]:
sub.count()

In [ ]:
pred1=pred.select(col('prediction').cast('integer'))

In [ ]:
pred1=pred1.toPandas()

In [ ]:
pred1

In [ ]:
sub.show()

In [ ]:
sub=sub.toPandas()

In [ ]:
sub['quality']=pred1['prediction']

**CONVERTING PREDICTIONS TO CSV FILE**

In [ ]:
sub.to_csv("wine_quality.csv",index=False)